In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [3]:
trans = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [11]:
train_data = torchvision.datasets.CIFAR10(
    root="./data", train=True, transform=trans, download=True
)

test_data = torchvision.datasets.CIFAR10(
    root="./data", train=False, transform=trans, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=32, shuffle=True, num_workers=2
)

# Pre-load data and labels to device
all_data, all_labels = [], []
for data, labels in train_loader:
    all_data.append(data.to(device))
    all_labels.append(labels.to(device))

# Combine all batches into a single tensor for data and labels
all_data = torch.cat(all_data)
all_labels = torch.cat(all_labels)

preloaded_dataset = torch.utils.data.TensorDataset(all_data, all_labels)
train_loader = torch.utils.data.DataLoader(preloaded_dataset, batch_size=32, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=32, shuffle=True, num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
img, label = train_data[0]

In [13]:
img.size()

torch.Size([3, 32, 32])

In [14]:
class_names = [
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [15]:
class NeuralNet(nn.Module):

    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
        self.pool = nn.MaxPool2d(2)  # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5)  # (24, 10, 10)
        # max pool -> (24, 5, 5)
        # flatten -> (24 * 5 * 5)
        self.dense1 = nn.Linear(24 * 5 * 5, 120)
        self.dense2 = nn.Linear(120, 84)
        self.dense3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = self.dense3(x)

        return x

In [16]:
net = NeuralNet()
net = net.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
for data, target in train_loader:
    data = data.to(device)
    target = target.to(device)

for epoch in range(30):
    print(f"epoch {epoch}")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_func(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()


    print(f"Loss: {running_loss / len(train_loader):.4f}")

epoch 0
Loss: 2.1858
epoch 1
Loss: 1.7166
epoch 2
Loss: 1.5242
epoch 3
Loss: 1.4039
epoch 4
Loss: 1.3139
epoch 5
Loss: 1.2327
epoch 6
Loss: 1.1623
epoch 7
Loss: 1.1000
epoch 8
Loss: 1.0419
epoch 9
Loss: 0.9903
epoch 10
Loss: 0.9438
epoch 11
Loss: 0.9083
epoch 12
Loss: 0.8680
epoch 13
Loss: 0.8327
epoch 14
Loss: 0.7999
epoch 15
Loss: 0.7687
epoch 16
Loss: 0.7425
epoch 17
Loss: 0.7155
epoch 18
Loss: 0.6867
epoch 19
Loss: 0.6635
epoch 20
Loss: 0.6418
epoch 21
Loss: 0.6175
epoch 22
Loss: 0.5963
epoch 23
Loss: 0.5751
epoch 24
Loss: 0.5529
epoch 25
Loss: 0.5327
epoch 26
Loss: 0.5129
epoch 27
Loss: 0.4973
epoch 28
Loss: 0.4724
epoch 29
Loss: 0.4550


In [26]:
torch.save(net.state_dict(), 'trained_net.pth')

In [27]:
# net = NeuralNet()
# net.load_state_dict(torch.load('trained_net.pth'))

In [31]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
	for data in test_loader:
		img, lb = data
		img, lb = img.to(device), lb.to(device)
		
		outputs = net(img)
		_, pred = torch.max(outputs, 1)
		total += lb.size(0)
		correct += (pred == lb).sum().item()

accuracy = 100 * correct / total

print(accuracy)

68.95


In [32]:
new_trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [33]:
def load_img(path):
    img = Image.open(path)
    img = new_trans(img)
    img = img.unsqueeze(0) # turn into batch
    return img

paths = [
    './dog.png', './plane.png'
]

imgs = [load_img(p) for p in paths]

In [36]:
net.eval()

with torch.no_grad():
	for img in imgs:
		outputs = net(img.to(device))
		_, pred = torch.max(outputs, 1)
		print(class_names[pred.item()])

bird
plane
